In [1]:
import train_yolo3

import mxnet as mx
import argparse
import os
import logging
import time
import warnings
import numpy as np
import mxnet as mx
from mxnet import nd
from mxnet import gluon
from mxnet import autograd
import gluoncv as gcv
gcv.utils.check_version('0.6.0')
from gluoncv import data as gdata
from gluoncv import utils as gutils
from gluoncv.model_zoo import get_model
from gluoncv.data.batchify import Tuple, Stack, Pad
from gluoncv.data.transforms.presets.yolo import YOLO3DefaultTrainTransform
from gluoncv.data.transforms.presets.yolo import YOLO3DefaultValTransform
from gluoncv.data.dataloader import RandomTransformDataLoader
from gluoncv.data import VOCDetection
from gluoncv.utils.metrics.voc_detection import VOC07MApMetric
from gluoncv.utils.metrics.coco_detection import COCODetectionMetric
from gluoncv.utils import LRScheduler, LRSequential

In [2]:
ctx = [mx.gpu()]

In [3]:
net_name = '_'.join(('yolo3', 'mobilenet0.25', 'custom'))

net = get_model(net_name, pretrained_base=True, classes=('cat', 'dog'), ctx=ctx)
net.initialize(force_reinit=False, ctx=ctx)
print('Initialized')

Initialized


/home/djn/opt/conda3/envs/mxnew/lib/python3.7/site-packages/mxnet/gluon/parameter.py:862: UserWarning: Parameter 'mobilenet0_conv0_weight' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
/home/djn/opt/conda3/envs/mxnew/lib/python3.7/site-packages/mxnet/gluon/parameter.py:862: UserWarning: Parameter 'mobilenet0_batchnorm0_gamma' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
/home/djn/opt/conda3/envs/mxnew/lib/python3.7/site-packages/mxnet/gluon/parameter.py:862: UserWarning: Parameter 'mobilenet0_batchnorm0_beta' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
/home/djn/opt/conda3/envs/mxnew/lib/python3.7/site-packages/mxnet/gluon/parameter.py:862: UserWarning: Parameter 'mobilenet0_batchnorm0_running_mean' is already initialized, ign

In [12]:
batch_size = 32

In [11]:
class petVOC(VOCDetection):
    CLASSES = ['cat', 'dog']
    def __init__(self, root=os.path.join('~','.mxnet','datasets','voc'), 
    splits=( (2007,'trainval'), (2012,'trainval')), transform=None, index_map=None, preload_label=True):
        
        super().__init__(root=root, splits=splits, transform=transform, index_map=index_map, preload_label=preload_label)

#%%
train_dataset = petVOC(root=os.path.join('~', 'Datasets', 'pets', 'dataset'), 
                                splits=((2019, 'train_val'),))
val_metric = VOC07MApMetric(iou_thresh=0.5, class_names=train_dataset.classes)

In [13]:
batchify_fn = Tuple(*([Stack() for _ in range(6)] + [Pad(axis=0, pad_val=-1) 
                                                    for _ in range(1)]))

In [17]:
train_loader = gluon.data.DataLoader(
            train_dataset.transform(YOLO3DefaultTrainTransform(168, 168, net)),
            batch_size, True, batchify_fn=batchify_fn, last_batch='rollover', num_workers=2)

val_batchify_fn = Tuple(Stack(), Pad(pad_val=-1))
val_loader = gluon.data.DataLoader(
        train_dataset.transform(YOLO3DefaultValTransform(168, 168)),
        batch_size, False, batchify_fn=val_batchify_fn, last_batch='keep', num_workers=2)

In [ ]:
def train(net, train_data, val_data, eval_metric, ctx, lr
          start_epoch, epochs, log_interval):
    """Training pipeline"""
    net.collect_params().reset_ctx(ctx)
    net._target_generator._label_smooth = True


    lr_decay_epoch = [5,10,20,35]
    lr_decay_epoch = [e - 0 for e in lr_decay_epoch]
    num_batches = args.num_samples // args.batch_size
    lr_scheduler = LRSequential([
        LRScheduler('linear', base_lr=0, target_lr=lr,
                    nepochs=0, iters_per_epoch=num_batches),
        LRScheduler('step', base_lr=lr,
                    nepochs=epochs - warmup_epochs,
                    iters_per_epoch=num_batches,
                    step_epoch=lr_decay_epoch,
                    step_factor=0.2, power=2),
    ])

    trainer = gluon.Trainer(
            net.collect_params(), 'sgd',
            {'wd': 0.0001, 'momentum': 0.9, 'lr_scheduler': lr_scheduler},
            kvstore='local', update_on_kvstore=(False if args.amp else None))

    # targets
    sigmoid_ce = gluon.loss.SigmoidBinaryCrossEntropyLoss(from_sigmoid=False)
    l1_loss = gluon.loss.L1Loss()

    # metrics
    obj_metrics = mx.metric.Loss('ObjLoss')
    center_metrics = mx.metric.Loss('BoxCenterLoss')
    scale_metrics = mx.metric.Loss('BoxScaleLoss')
    cls_metrics = mx.metric.Loss('ClassLoss')

    # set up logger
    logging.basicConfig()
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)
    log_file_path = args.save_prefix + '_train.log'
    log_dir = os.path.dirname(log_file_path)
    if log_dir and not os.path.exists(log_dir):
        os.makedirs(log_dir)
    fh = logging.FileHandler(log_file_path)
    logger.addHandler(fh)
    logger.info(args)
    logger.info('Start training from [Epoch {}]'.format(start_epoch))
    best_map = [0]
    for epoch in range(start_epoch, epochs):
        tic = time.time()
        btic = time.time()
        mx.nd.waitall()
        net.hybridize()
        for i, batch in enumerate(train_data):
            data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
            # objectness, center_targets, scale_targets, weights, class_targets
            fixed_targets = [gluon.utils.split_and_load(batch[it], ctx_list=ctx, batch_axis=0) for it in range(1, 6)]
            gt_boxes = gluon.utils.split_and_load(batch[6], ctx_list=ctx, batch_axis=0)
            sum_losses = []
            obj_losses = []
            center_losses = []
            scale_losses = []
            cls_losses = []
            with autograd.record():
                for ix, x in enumerate(data):
                    obj_loss, center_loss, scale_loss, cls_loss = net(x, gt_boxes[ix], *[ft[ix] for ft in fixed_targets])
                    sum_losses.append(obj_loss + center_loss + scale_loss + cls_loss)
                    obj_losses.append(obj_loss)
                    center_losses.append(center_loss)
                    scale_losses.append(scale_loss)
                    cls_losses.append(cls_loss)

                autograd.backward(sum_losses)
            trainer.step(batch_size)
            
            obj_metrics.update(0, obj_losses)
            center_metrics.update(0, center_losses)
            scale_metrics.update(0, scale_losses)
            cls_metrics.update(0, cls_losses)
            if log_interval and not (i + 1) % args.log_interval:
                name1, loss1 = obj_metrics.get()
                name2, loss2 = center_metrics.get()
                name3, loss3 = scale_metrics.get()
                name4, loss4 = cls_metrics.get()
                logger.info('[Epoch {}][Batch {}], LR: {:.2E}, Speed: {:.3f} samples/sec, {}={:.3f}, {}={:.3f}, {}={:.3f}, {}={:.3f}'.format(
                        epoch, i, trainer.learning_rate, args.batch_size/(time.time()-btic), name1, loss1, name2, loss2, name3, loss3, name4, loss4))
            btic = time.time()

        name1, loss1 = obj_metrics.get()
        name2, loss2 = center_metrics.get()
        name3, loss3 = scale_metrics.get()
        name4, loss4 = cls_metrics.get()
        logger.info('[Epoch {}] Training cost: {:.3f}, {}={:.3f}, {}={:.3f}, {}={:.3f}, {}={:.3f}'.format(
                epoch, (time.time()-tic), name1, loss1, name2, loss2, name3, loss3, name4, loss4))
        if not (epoch + 1) % args.val_interval:
                # consider reduce the frequency of validation to save time
            map_name, mean_ap = validate(net, val_data, ctx, eval_metric)
            val_msg = '\n'.join(['{}={}'.format(k, v) for k, v in zip(map_name, mean_ap)])
            logger.info('[Epoch {}] Validation: \n{}'.format(epoch, val_msg))
            current_map = float(mean_ap[-1])
        else:
            current_map = 0.
        save_params(net, best_map, current_map, epoch, args.save_interval, args.save_prefix)